In [282]:
p_storage_dir = "../../storage"

In [283]:
import pandas as pd
import numpy as np
from pathlib import Path

IFG_PRODUZ_STORAGE_DIR = Path(f"{p_storage_dir}/ifg_produz")

## Tabela Intermediária

![Tabela Intermediária](./assets/images/tabela_intermediaria_grupos_de_pesquisa.png)

In [284]:
intermediaria = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "extracted/curriculo_grupos.csv", delimiter=";", dtype={'curriculo_id': 'Int64'})

In [285]:
intermediaria.head()

,id_curriculo_linha_grupo,curriculo_id,grupos_id,linhas_id
0,1,888,1,1
1,2,1438,1,1
2,3,973,1,1
3,4,1324,1,1
4,5,39,1,1


## Linhas de Pesquisa

In [286]:
linhas_de_pesquisa = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "extracted/linhas_de_pesquisa.csv", delimiter=";")

linhas_de_pesquisa.head(n=5)

,id_linha,nome_da_linha
0,1,"Ambiente, Saúde, Qualidade de Vida e Sociedade"
1,2,Estudos interdisciplinares em Infecções Relaci...
2,3,Formação humana e cultura corporal
3,4,"Política educacional, currículo e trabalho doc..."
4,5,Processo educativo e trabalho pedagógico


In [287]:
linhas_de_pesquisa = linhas_de_pesquisa.rename(columns={
    'id_linha': 'codigo',
    'nome_da_linha': 'titulo',
})

linhas_de_pesquisa.sample(n=5)

,codigo,titulo
146,147,"Fundamentos, metodologias e recursos para a Ed..."
19,20,Estudo da flexão em elementos lineares de conc...
211,212,Logística e Transporte de Cargas
169,170,Planejamento Integrado dos Recursos Hídricos
9,10,Linguística Aplicada e o Ensino de Língua Mate...


In [288]:
if not (linhas_de_pesquisa['titulo'] == linhas_de_pesquisa['titulo'].str.strip()).all():
    linhas_de_pesquisa['titulo'] = linhas_de_pesquisa['titulo'].str.strip()

### Inserir em uma nova coluna com todos os currículos que estão ligados com cada linha de pesquisa

In [289]:
list(intermediaria[intermediaria['linhas_id'] == 44]['curriculo_id'].dropna().unique())

[759, 238]

In [290]:
def create_df_curriculos_to_linha_de_pesquisa(intermediate: pd.DataFrame, linhas_de_pesquisa: pd.DataFrame) -> pd.DataFrame:
    final_map = {}
    
    for codigo_linha_de_pesquisa in linhas_de_pesquisa['codigo'].unique():
        codigos_curriculos = list(intermediaria[intermediaria['linhas_id'] == codigo_linha_de_pesquisa]['curriculo_id'].dropna().unique())

        if len(codigos_curriculos) > 0:
            final_map[codigo_linha_de_pesquisa] = codigos_curriculos

    return final_map

In [291]:
linha_de_pesquisa_to_curriculos_map = create_df_curriculos_to_linha_de_pesquisa(intermediaria, linhas_de_pesquisa)

In [292]:
def set_curriculos_in_linha_de_pesquisa(row, map):
    codigo_linha_de_pesquisa = row['codigo']
    
    if codigo_linha_de_pesquisa in map.keys():
        curriculos = map[row['codigo']]
        return curriculos

In [293]:
linhas_de_pesquisa['codigos_curriculos'] = linhas_de_pesquisa.apply(set_curriculos_in_linha_de_pesquisa, axis=1,
                                                                    args=(linha_de_pesquisa_to_curriculos_map,))

In [294]:
linhas_de_pesquisa.head()

,codigo,titulo,codigos_curriculos
0,1,"Ambiente, Saúde, Qualidade de Vida e Sociedade","[888, 1438, 973, 1324, 39, 386, 509, 4, 1388, ..."
1,2,Estudos interdisciplinares em Infecções Relaci...,"[1324, 386, 509, 4, 1388]"
2,3,Formação humana e cultura corporal,"[1204, 711, 827, 600, 130]"
3,4,"Política educacional, currículo e trabalho doc...","[827, 130, 1146, 1073]"
4,5,Processo educativo e trabalho pedagógico,"[915, 1657, 1419, 1334, 1795, 471]"


## Grupos de Pesquisa

In [295]:
grupos_de_pesquisa = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "extracted/grupos_de_pesquisa.csv", delimiter=";")

grupos_de_pesquisa.head(n=5)

,id_grupo,nome_grupo,situacao_do_grupo,ano_formacao,lider_do_grupo,link_do_grupo,linha_de_pesquisa,situacao_do_grupo_por_criterios
0,1,Núcleo de Estudos e Pesquisas em Ciências da Vida,Certificado,2020,Patricia Carvalho de Oliveira,http://dgp.cnpq.br/dgp/espelhogrupo/633526,NaN,Certificado
1,2,Núcleo de estudos e pesquisas socicríticas em ...,Em preenchimento,2020,Fernando Henrique Silva Carneiro,http://dgp.cnpq.br/dgp/espelhogrupo/590146,NaN,Certificado
2,3,Grupo de Pesquisas em Artes e Audiovisual,Em preenchimento,2020,Adérito Schneider Alencar e Távora,http://dgp.cnpq.br/dgp/espelhogrupo/607747,NaN,Certificado
3,4,Núcleo Multicampi de Pesquisas e Estudos em Li...,Certificado,2011,Mabel Pettersen Prudente,http://dgp.cnpq.br/dgp/espelhogrupo/33150,NaN,Pendente
4,5,"Núcleo de Estudos de Cultura, Linguagens e sua...",Certificado,2013,Rangel Gomes Godinho,http://dgp.cnpq.br/dgp/espelhogrupo/377570,NaN,Pendente


In [296]:
grupos_de_pesquisa = grupos_de_pesquisa.rename(columns={
    'id_grupo': 'codigo',
    'nome_grupo': 'nome',
    'situacao_do_grupo': 'situacao',
    'situacao_do_grupo_por_criterios': 'situacao_por_criterios',
    'lider_do_grupo': 'nome_lider',
    'link_do_grupo': 'link'
})

In [297]:
grupos_de_pesquisa.head(n=5)

,codigo,nome,situacao,ano_formacao,nome_lider,link,linha_de_pesquisa,situacao_por_criterios
0,1,Núcleo de Estudos e Pesquisas em Ciências da Vida,Certificado,2020,Patricia Carvalho de Oliveira,http://dgp.cnpq.br/dgp/espelhogrupo/633526,NaN,Certificado
1,2,Núcleo de estudos e pesquisas socicríticas em ...,Em preenchimento,2020,Fernando Henrique Silva Carneiro,http://dgp.cnpq.br/dgp/espelhogrupo/590146,NaN,Certificado
2,3,Grupo de Pesquisas em Artes e Audiovisual,Em preenchimento,2020,Adérito Schneider Alencar e Távora,http://dgp.cnpq.br/dgp/espelhogrupo/607747,NaN,Certificado
3,4,Núcleo Multicampi de Pesquisas e Estudos em Li...,Certificado,2011,Mabel Pettersen Prudente,http://dgp.cnpq.br/dgp/espelhogrupo/33150,NaN,Pendente
4,5,"Núcleo de Estudos de Cultura, Linguagens e sua...",Certificado,2013,Rangel Gomes Godinho,http://dgp.cnpq.br/dgp/espelhogrupo/377570,NaN,Pendente


### Verificando valores nulos

In [298]:
grupos_de_pesquisa.isna().sum()

codigo                     0
nome                       0
situacao                   0
ano_formacao               0
nome_lider                 0
link                       0
linha_de_pesquisa         76
situacao_por_criterios     0
dtype: int64

In [299]:
grupos_de_pesquisa['linha_de_pesquisa'].shape

(76,)

### Ligar as Linhas de Pesquisa ao Grupo de Pesquisa

In [300]:
intermediaria.sample(n=5)

,id_curriculo_linha_grupo,curriculo_id,grupos_id,linhas_id
381,383,139,28,98
454,456,1363,33,118
214,215,558,11,50
1,2,1438,1,1
911,913,870,70,255


In [301]:
grupos_de_pesquisa = grupos_de_pesquisa.rename(columns={'linha_de_pesquisa': 'codigos_linhas_de_pesquisa'})

In [302]:
grupos_de_pesquisa.columns

Index(['codigo', 'nome', 'situacao', 'ano_formacao', 'nome_lider', 'link',
       'codigos_linhas_de_pesquisa', 'situacao_por_criterios'],
      dtype='object')

In [303]:
def find_and_set_research_lines_for_research_groups(row, intermediaria: pd.DataFrame):
    research_lines_codes = list(intermediaria[intermediaria['grupos_id'] == row['codigo']]['linhas_id'].unique())
    row['codigos_linhas_de_pesquisa'] = research_lines_codes

    return row

In [304]:
grupos_de_pesquisa = grupos_de_pesquisa.apply(find_and_set_research_lines_for_research_groups, args=(intermediaria,), axis=1)

In [305]:
grupos_de_pesquisa.head()

,codigo,nome,situacao,ano_formacao,nome_lider,link,codigos_linhas_de_pesquisa,situacao_por_criterios
0,1,Núcleo de Estudos e Pesquisas em Ciências da Vida,Certificado,2020,Patricia Carvalho de Oliveira,http://dgp.cnpq.br/dgp/espelhogrupo/633526,"[1, 2]",Certificado
1,2,Núcleo de estudos e pesquisas socicríticas em ...,Em preenchimento,2020,Fernando Henrique Silva Carneiro,http://dgp.cnpq.br/dgp/espelhogrupo/590146,"[3, 4, 5]",Certificado
2,3,Grupo de Pesquisas em Artes e Audiovisual,Em preenchimento,2020,Adérito Schneider Alencar e Távora,http://dgp.cnpq.br/dgp/espelhogrupo/607747,"[6, 7, 8]",Certificado
3,4,Núcleo Multicampi de Pesquisas e Estudos em Li...,Certificado,2011,Mabel Pettersen Prudente,http://dgp.cnpq.br/dgp/espelhogrupo/33150,"[9, 10]",Pendente
4,5,"Núcleo de Estudos de Cultura, Linguagens e sua...",Certificado,2013,Rangel Gomes Godinho,http://dgp.cnpq.br/dgp/espelhogrupo/377570,"[11, 12, 13]",Pendente


### Inserir uma coluna com o ID do currículo de cada líder

In [306]:
from text_processors import normalizar_string

In [307]:
curriculos = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "extracted/curriculo.csv", delimiter=";")
curriculos = curriculos.loc[:, ['id_curriculo', 'nome_completo']]

In [308]:
curriculos['nome_completo_normalizado'] = curriculos['nome_completo'].apply(normalizar_string)

In [309]:
curriculos.sample(n=3)

,id_curriculo,nome_completo,nome_completo_normalizado
1607,170,Katia Pereira Fonseca,katia pereira fonseca
1757,665,Adrielly Melo Borges,adrielly melo borges
1742,617,Angelita Duarte da Silva,angelita duarte da silva


In [310]:
linhas_de_pesquisa.sample(n=3)

,codigo,titulo,codigos_curriculos
119,120,Processamento de Sinais Biológicos,"[1756, 1071]"
235,236,Desenvolvimento de Software para Sistemas Elét...,[1406]
110,111,Construção Civil,"[1287, 142, 1748, 752]"


In [311]:
intermediaria.sample(n=3)

,id_curriculo_linha_grupo,curriculo_id,grupos_id,linhas_id
95,96,1081,6,28
850,852,<NA>,65,241
975,977,833,76,278


In [312]:
def find_and_set_research_group_leader_code(row, curriculos: pd.DataFrame, linhas_de_pesquisa: pd.DataFrame, intermediaria: pd.DataFrame):
    # rg = research group
    rg_leader_normalized = normalizar_string(row['nome_lider'])
    rg_code = row['codigo']

    found_leader_resume = curriculos[curriculos['nome_completo_normalizado'] == rg_leader_normalized]

    if len(found_leader_resume) > 1:
        print(f'Não foi possível encontrar o código correto para o líder do grupo com ID: {rg_code}. Foi encontrado mais de um resultado na busca realizada.')
        return
    elif len(found_leader_resume) == 0:
        print(f'Não foi possível encontrar o líder do grupo com ID: {rg_code}. Não foram encontrados registros.')
        return
    
    found_leader_code = found_leader_resume['id_curriculo'].iloc[0]
    found_leader_research_lines = intermediaria[(intermediaria['curriculo_id'] == found_leader_code) & 
    (intermediaria['grupos_id'] == row['codigo'])]
    
    if len(found_leader_research_lines) == 0:
        print(f"Não foi possível ligar o líder do grupo com ID: {rg_code}. O líder do grupo não está ligado a nenhuma linha de pesquisa do grupo.")
        return

    row['codigo_lider'] = found_leader_code

    return row

In [313]:
grupos_de_pesquisa = grupos_de_pesquisa.apply(find_and_set_research_group_leader_code, args=(curriculos, linhas_de_pesquisa, intermediaria), axis=1)

Não foi possível ligar o líder do grupo com ID: 27. O líder do grupo não está ligado a nenhuma linha de pesquisa do grupo.


In [314]:
grupos_de_pesquisa['codigo_lider'] = grupos_de_pesquisa['codigo_lider'].astype('Int64')

### Colocar o código dos participantes do grupo em uma coluna

In [315]:
intermediaria.sample(n=5)

,id_curriculo_linha_grupo,curriculo_id,grupos_id,linhas_id
845,847,1097,65,238
322,324,1040,20,80
654,656,1792,51,177
394,396,1467,29,100
917,919,211,71,258


In [316]:
def find_and_set_group_member_codes(row, intermediaria: pd.DataFrame):
    group_members_codes = list(intermediaria[intermediaria['grupos_id'] == row['codigo']]['curriculo_id'].dropna().unique())

    row['codigo_membros'] = group_members_codes

    return row

In [317]:
grupos_de_pesquisa = grupos_de_pesquisa.apply(find_and_set_group_member_codes, args=(intermediaria,), axis=1)

## Alunos

In [318]:
alunos = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "extracted/alunos.csv", delimiter=";")

alunos.head(n=5)

,id_aluno,nome_aluno,id_grupos_de_pesquisa_id
0,1,Amanda Sousa de Queiros,1
1,2,Ellen Kelrye Ferreira Reis,1
2,3,Janailza Barbosa da Silva,1
3,4,Jariele Ferreira de Oliveira,1
4,5,Maisa Cecília da Silva Reis,1


In [319]:
alunos = alunos.rename(columns={
    'id_aluno': 'codigo',
    'nome_aluno': 'nome',
    'id_grupos_de_pesquisa_id': 'codigo_grupo_de_pesquisa'
})

In [320]:
alunos.columns

Index(['codigo', 'nome', 'codigo_grupo_de_pesquisa'], dtype='object')

## Exportar

In [321]:
export_path = Path(IFG_PRODUZ_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

linhas_de_pesquisa.to_csv(export_path / "linhas_de_pesquisa.csv", sep=";", index=False)
grupos_de_pesquisa.to_csv(export_path / "grupos_de_pesquisa.csv", sep=";", index=False)
alunos.to_csv(export_path / "alunos.csv", sep=";", index=False)